In [ ]:
from init_notebook import *
from src.train.experiment import load_experiment_trainer

In [ ]:
trainer = load_experiment_trainer("../experiments/img2img/shinytubes/diffusion/shinytubes-spikes-diffusion.yml", device="cuda")
trainer.model.eval()

In [ ]:
source_batch, target_batch, amount_batch = next(iter(trainer.data_loader))

In [ ]:
VF.to_pil_image(make_grid(source_batch.clamp(0, 1)))

In [ ]:
with torch.no_grad():
    recon_batch = trainer.model(source_batch.to(trainer.device), amount_batch.to(trainer.device))

VF.to_pil_image(make_grid(recon_batch.clamp(0, 1)))

In [ ]:
with torch.no_grad():
    recon_batch = trainer.model(
        source_batch.to(trainer.device), 
        torch.ones(source_batch.shape[0]).to(trainer.device) * .1,
    )

VF.to_pil_image(make_grid(recon_batch.clamp(0, 1)))

In [ ]:
with torch.no_grad():
    NUM = 8
    x = source_batch.to(trainer.device)
    noise = torch.rand_like(x)
    for i in range(NUM):
        t = (NUM - i) / NUM
        x = trainer.model(
            x, 
            torch.ones(x.shape[0]).to(trainer.device) * .5,
        ).clamp(0, 1)
        if i + 1 < NUM:
            x = x + t * (noise - x) * .2

VF.to_pil_image(resize(make_grid(x.clamp(0, 1)), 3))

In [ ]:
from PIL import ImageFont, ImageDraw
scale = 1
font = ImageFont.truetype(
    "/home/bergi/.local/share/fonts/LEMONMILK-LIGHTITALIC.OTF", 20 * scale
    #"/home/bergi/.local/share/fonts/LEMONMILK-MEDIUMITALIC.OTF", 20 * scale
    #"/home/bergi/.local/share/fonts/unscii-16-full.ttf", 25 * scale
    #"/usr/share/fonts/truetype/open-sans/OpenSans-ExtraBold.ttf", 25 * scale
    #"/usr/share/fonts/truetype/dejavu/DejaVuSerif.ttf", 25 * scale
)
image = PIL.Image.new("RGB", (250 * scale, 40 * scale))
draw = ImageDraw.ImageDraw(image)
draw.text((30, 7), "h e l l o  w o r l d", font=font, fill=(255, 255, 255))
image = VF.to_tensor(image)
VF.to_pil_image(image)

In [ ]:
with torch.no_grad():
    NUM = 10
    x = image.unsqueeze(0).to(trainer.device)
    noise = torch.rand_like(x)
    noise = noise.mean(dim=-3, keepdim=True).repeat(1, 3, 1, 1)
    out_batch = []
    for i in range(NUM):
        t = (NUM - i) / NUM
        x = trainer.model(
            x, 
            torch.ones(x.shape[0]).to(trainer.device) * .45 * (.8 + .2*t),
        ).clamp(0, 1)
        out_batch.append(x[0])
        if i + 1 < NUM:
            #noise = torch.rand_like(x)
            x = x + (noise - x) * .4 #* t
            #out_batch.append(x[0])

VF.to_pil_image(resize(make_grid(out_batch, nrow=1), 2))